In [1]:
import matplotlib.pyplot as plt
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.svm
import sklearn as sk
import pandas as pd
import numpy as np
%matplotlib inline

lower = False
feature = 'title'
#feature = 'tags'
#feature = 'description'
region, language = 'US', 'english'   # 
#region, language = 'KR', 'korean'    # doesn't work
#region, language = 'MX', 'spanish'   # doesn't work
#region, language = 'CA', 'english'   #
#region, language = 'DE', 'german'    #
#region, language = 'FR', 'french'    #

def load_region_data(region: str) -> pd.DataFrame:
    df = pd.read_csv('./archive/{}videos.csv'.format(region))

    # Drop unused columns
    df = df.drop(['thumbnail_link', 'video_id'], axis=1)
    df = df.dropna()

    # Enrich data
    df['publish_hour'] = pd.to_datetime(df['publish_time']).dt.hour
    df['comments_disabled'] = df['comments_disabled'].apply(lambda row: 0 if row == False else 1)
    df['ratings_disabled'] = df['ratings_disabled'].apply(lambda row: 0 if row == False else 1)
    df['video_error_or_removed'] = df['video_error_or_removed'].apply(lambda row: 0 if row == False else 1)
    df['like_dislike'] = df[['likes', 'dislikes']].apply(lambda row: (row['likes'] / np.sum([row['likes'], row['dislikes']]))*100, axis=1)
    df['tags'] = df['tags'].apply(lambda row: ' '.join( i.strip('""') for i in row.split('|') ))
    df['anti_participation'] = df[['comments_disabled', 'ratings_disabled']].apply(lambda row: row.sum(), axis=1)
    df['morning_video'] = df['publish_hour'].apply(lambda t: 1 if t <= 12 else 0)

    # Transform category_id to label
    category_le = sk.preprocessing.LabelEncoder()
    category_le.fit(df['category_id'])
    df['category_id'] = category_le.transform(df['category_id'])
    
    #views = df['views'].to_numpy().reshape((-1,1))
    #view_scaler = sk.preprocessing.MinMaxScaler()
    #df['views'] = view_scaler.fit_transform(views)*100
    
    df = df.dropna()
    
    return df, len(category_le.classes_)

df, num_classes = load_region_data(region)
df.head()

<ipython-input-1-fad7ad4d6fca>:33: RuntimeWarning: invalid value encountered in long_scalars
  df['like_dislike'] = df[['likes', 'dislikes']].apply(lambda row: (row['likes'] / np.sum([row['likes'], row['dislikes']]))*100, axis=1)


,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,description,publish_hour,like_dislike,anti_participation,morning_video
0,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,7,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,0,0,0,SHANTELL'S CHANNEL - https://www.youtube.com/s...,17,95.096953,0,0
1,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,9,2017-11-13T07:30:00.000Z,last week tonight trump presidency last week t...,2418783,97185,6146,12703,0,0,0,"One year after the presidential election, John...",7,94.052124,0,1
2,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,8,2017-11-12T19:05:24.000Z,racist superman rudy mancuso king bach racist ...,3191434,146033,5339,8181,0,0,0,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,19,96.472928,0,0
3,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,9,2017-11-13T11:00:04.000Z,rhett and link gmm good mythical morning rhett...,343168,10172,666,2146,0,0,0,Today we find out if Link is a Nickelback amat...,11,93.854955,0,1
4,17.14.11,I Dare You: GOING BALD!?,nigahiga,9,2017-11-12T18:01:41.000Z,ryan higa higatv nigahiga i dare you idy rhpc ...,2095731,132235,1989,17518,0,0,0,I know it's been a while since we did this sho...,18,98.518149,0,0


In [9]:
X = df[['category_id', 'views', 'likes', 'dislikes', 'like_dislike', 'comment_count', 'anti_participation']].to_numpy()
Y = df[['morning_video']].to_numpy()

xtr, xts, ytr, yts = sk.model_selection.train_test_split(X, Y, shuffle=True, test_size=0.5)

In [ ]:
#svm = sk.model_selection.GridSearchCV(
#    estimator=sk.svm.SVC(C=1000, gamma=0.0001),
#    param_grid={'C': [1, 10e1, 10e2], 'kernel': ['poly'], 'gamma': [0.01, 0.001, 0.0001]},
#    n_jobs=8,
#    verbose=5,
#)
svm = sk.svm.SVC(C=1000, gamma='auto', kernel='poly', verbose=5)
print()
print(svm.fit(xtr, ytr))
print(svm.score(xts, yts))


[LibSVM]

/home/jc/nyu/ml/proj/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
